In [1]:
# --- 3. Установка проекта rank_images_project из ветки feature/blip_caption-metric ---
# Клонируем репозиторий
!git clone https://github.com/Mike030668/rank_images_project.git
# Переходим в папку проекта
%cd rank_images_project
# Переключаемся на нужную ветку
!git checkout feature/blip_caption-metric
# Устанавливаем проект и зависимости в режиме редактирования
# Указываем дополнительный индекс для PyTorch и устанавливаем caption-зависимости
!pip install --pre \
  --extra-index-url https://download.pytorch.org/whl/nightly/cu121 \
  -e .[caption]

print("Установка зависимостей завершена.")

Cloning into 'rank_images_project'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 221 (delta 114), reused 204 (delta 97), pack-reused 0 (from 0)
Receiving objects: 100% (221/221), 3.09 MiB | 7.46 MiB/s, done.
Resolving deltas: 100% (114/114), done.
/content/rank_images_project
Branch 'feature/blip_caption-metric' set up to track remote branch 'feature/blip_caption-metric' from 'origin'.
Switched to a new branch 'feature/blip_caption-metric'
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/nightly/cu121
Obtaining file:///content/rank_images_project
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

Установка зависимостей завершена.


In [1]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00


# Настройка логирования и тестирование

In [2]:
# Ячейка 3: Настройка логирования и тестирование
import sys
from pathlib import Path
import logging

# --- Настройка логирования ---
# Включаем DEBUG-логи для нашего проекта
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
print("Логирование настроено.")
# -----------------------------

# --- Добавление пути к проекту ---
project_root = Path('/content/rank_images_project')
sys.path.insert(0, str(project_root / "src"))
print(f"Путь к проекту '{project_root}/src' добавлен в sys.path")
# -------------------------------

# --- Импорт и загрузка моделей ---
from rank_images.models import load_models
from rank_images.ranking import rank_folder

print("Начинаю загрузку моделей...")
try:
    # Загружаем ВСЕ модели (можно передать список, если нужно оптимизировать)
    load_models() # <-- Передаем None, чтобы загрузить все
    print("Загрузка моделей завершена.")
except Exception as e:
    print(f"Ошибка при загрузке моделей: {e}")
    import traceback
    traceback.print_exc()
# --------------------------------

# --- Тестирование ранжирования ---
test_images_path = Path("/content/rank_images_project/data/demo_images")
prompts_path = test_images_path / "prompts.json"
# Создаем конфигурацию пайплайна "на лету" для теста
# Включаем blip_cap и blip2_cap
test_pipeline_config = {
    "pipeline": {
        "enabled_metrics": ["iqa", "blip_cap", "blip2_cap"], # <-- Включаем нужные метрики
        "default_weights": {
            "gamma": 0.4,
            "zeta": 0.3, # <-- Вес для blip_cap
            "theta": 0.3 # <-- Вес для blip2_cap
        }
    },
    "processing": {
        "chunk_size": 128
    }
}

print("Начинаю процесс ранжирования...")
try:
    result_df = rank_folder(
        img_dir=test_images_path,
        prompts_in=str(prompts_path),
        # Передаем веса из тестового конфига
        gamma=test_pipeline_config["pipeline"]["default_weights"]["gamma"],
        zeta=test_pipeline_config["pipeline"]["default_weights"]["zeta"],
        theta=test_pipeline_config["pipeline"]["default_weights"]["theta"],
        # Передаем сам конфиг пайплайна
        pipeline_config= test_pipeline_config
    )
    print("Ранжирование завершено.")
    print(result_df.head(10)) # Показываем все строки результата
except Exception as e:
    print(f"Ошибка во время ранжирования: {e}")
    import traceback
    traceback.print_exc()
# ----------------------------------

Логирование настроено.
Путь к проекту '/content/rank_images_project/src' добавлен в sys.path
Начинаю загрузку моделей...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

Ранжирование завершено.
      image       iqa  blip_cap  blip2_cap  iqa_norm  blip_cap_norm  \
0   999.png  0.915174  0.840817   0.845887  0.837018       0.342900   
1   888.png  0.943873  0.844080   0.840816  1.174593       1.171980   
2   111.png  0.912280  0.846371   0.840070  0.802977       1.754367   
3  1010.png  0.871476  0.841084   0.841724  0.323003       0.410756   
4   777.png  0.799303  0.838305   0.845887 -0.525943      -0.295469   
5   222.png  0.847685  0.842598   0.839920  0.043162       0.795480   
6   333.png  0.918014  0.835829   0.839920  0.870418      -0.924539   
7   555.png  0.834715  0.835829   0.841365 -0.109397      -0.924539   
8   666.png  0.666524  0.835829   0.846259 -2.087787      -0.924539   
9   444.png  0.731113  0.833933   0.841365 -1.328043      -1.406398   

   blip2_cap_norm     total  
0        1.432136  0.867318  
1       -0.604754  0.640005  
2       -0.904227  0.576233  
3       -0.240043  0.180415  
4        1.432136  0.130623  
5       -0.964